In [1]:
import pandas as pd

In [2]:
import datacompy

In [3]:
from excel_maker import Support

In [4]:
from postgress_config import PostgressDB

In [5]:
import time

In [6]:
from oracle_config import Ora

In [7]:
import numpy as np

In [8]:
sup = Support()

In [9]:
main_dataframe = pd.read_excel(r'resources\main_dataframe.xlsx')

In [10]:
#df = pd.read_excel(r'resources\doc.xlsx')
#d = datacompy.Compare(main_dataframe,df,join_columns='PATIENT_NAME')
#sup.excel_generator(excel_data=d.df1_unq_rows, page_name="master")

In [11]:
#main_dataframe = sup.negative_to_postive_entry_check(main_dataframe)

In [17]:
sup.excel_generator(excel_data=main_dataframe, page_name="main_dataframe")

'D:\\Python Projects\\doctors_payout\\Scripts\\doctors_payout/main_dataframe-12-May-2023-14-24-28-025260.xlsx'